In [1]:
import pandas as pd
import numpy as np

#Load data
df = pd.read_csv("DataLayer_Results_New_Gui.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#Create empty DataFrame
df_transform = pd.DataFrame()
#save for reference
length = len(df)
#List for leaks naming
names = ["_2013", "_2014", "_2015", "_2016", "_2017", "_H_2013", "_H_2014", 
         "_H_2015", "_H_2016", "_H_2017"]

#Iterate through columns (up to SERV_MECHFIT_LEAKS), check to add, drop, or transform
for column in df.columns[:df.columns.get_loc("SERV_MECHFIT_LEAKS")+1]:
    
    #Drop column if only 1 unique value
    if df[column].nunique() == 1:
        pass #do nothing
    
    #Transform column with Y/N to 1/0
    elif (('N' in list(df[column])) & ('Y' in list(df[column]))):
        df[column][df[column] == 'Y'] = 1
        df[column][df[column] == 'N'] = 0
        df_transform[column] = df[column]
        
    #If column ends with LEAKS
    elif column[-5:] == "LEAKS":
        
        #Add column if it's a single int
        if np.issubdtype(df[column].dtype, np.number):
            df_transform[column] = df[column]
            
        #Don't add if it contains less than 5 non-zero strings
        elif (length - list(df[column]).count('0,0,0,0,0,0,0,0,0,0') <= 5):
            pass
        
        #Else split into 10 columns and add
        else:
            df_transform[[column + name for name in names]] = pd.DataFrame(list(df[column].str.split(',')))
                
    #Add all other columns
    else:
        df_transform[column] = df[column]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
#Fix columns with mixed datatypes and null values

#Replace nans in COAL_MINE with "Not Coal Mine"
df_transform["COAL_MINE"] = df["COAL_MINE"].fillna("Not Coal Mine")

#Remove commas from SYSTEM_NUM
df_transform["SYSTEM_NUM"] = df_transform["SYSTEM_NUM"].str.replace(",","")

#Remove commas from CP_DAYS_DEFICIENT
df_transform["CP_DAYS_DEFICIENT"] = df_transform["CP_DAYS_DEFICIENT"].str.replace(",","")

#Remove commas from POP_DENSITY
df_transform["POP_DENSITY"] = df_transform["POP_DENSITY"].str.replace(",","")

#Remove commas from RISK_AREA_FT
df_transform["RISK_AREA_FT"] = df_transform["RISK_AREA_FT"].str.replace(",","")

#Remove commas from DECADE
df_transform["DECADE"] = df_transform["DECADE"].str.replace(",","")

In [4]:
#Create ClASS column from FACILITY_CLASS and PRESS_CLASS

scenarios = [('DISTRIBUTION','FEEDER','HPD'),
             ('DISTRIBUTION','INTER CITY','HPD'),
             ('DISTRIBUTION','INTERMEDIATE 1','DISTRIBUTION'),
             ('DISTRIBUTION','INTERMEDIATE 2','DISTRIBUTION'),
             ('DISTRIBUTION','LOW','DISTRIBUTION'),
             ('SERVICE','FEEDER','HPD'),
             ('SERVICE','INTER CITY','HPD'),
             ('SERVICE','INTERMEDIATE 1','DISTRIBUTION'),       
             ('SERVICE','INTERMEDIATE 2','DISTRIBUTION'),
             ('SERVICE','LOW','DISTRIBUTION'),
             ('STORAGE','FEEDER','TRANSMISSION'),
             ('STORAGE','INTER CITY','TRANSMISSION'),
             ('TRANSMISSION','FEEDER','TRANSMISSION'),
             ('TRANSMISSION','INTER CITY','TRANSMISSION')]

for scenario in scenarios:
    a, b, c = scenario
    df_transform.loc[((df_transform['FACILITY_CLASS'] == a) & (df_transform['PRESS_CLASS'] == b)), "CLASS"] = c
    
#Drop FACILITY_CLASS and PRESS_CLASS
df_transform.drop(['FACILITY_CLASS','PRESS_CLASS'], axis = 1, inplace = True)

In [5]:
#Set RISK_AREA as index
df_transform = df_transform.set_index('RISK_AREA')

In [6]:
#Write cleaned data set to csv
df_transform.to_csv('Gas_Asset_Data_Cleaned.csv')